In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import six
import os

# Load in the stored data

In [2]:
DATA_FILE = 'data/four_players_data.csv'

In [3]:
df_all = pd.read_csv(DATA_FILE)
print('Original sample size', len(df_all))

Original sample size 6714


# Helper methods

In [4]:
def all_players(key):
    return ['pos_{}{}'.format(i, key) for i in range(1,5)]

In [5]:
def field_search(key, search_df=df_all):
    return {_ for _ in search_df if key in _}

In [6]:
def get_faction_df(df, faction):
    df_list = []
    for i in range(1,5) :
        col = 'pos_{}{}'.format(i, '_faction')
        pos_df = df.loc[df[col] == faction]
        col_dict = {}
        for pos_field in field_search('pos_{}'.format(i),df):
            target_field = 'target' + pos_field[5:]
            col_dict[target_field] = pos_df[pos_field]
        target_df = pd.concat(col_dict, axis=1)
        target_df = pd.concat([pos_df, target_df], axis=1) 
        df_list.append(target_df)
    df_faction = pd.concat(df_list)
    # display(df_faction['target_score'])
    df_faction['target_score'] = df_faction['target_score'] + df_faction['target_bid']
    # display(df_faction['target_score'])
    # display(df_faction['target_score_bid'].value_counts())
    return df_faction

In [7]:
def getTableHTML(df):
    
    """
    From https://stackoverflow.com/a/49687866/2007153
    
    Get a Jupyter like html of pandas dataframe
    
    """

    
    styles = [
        #table properties
        dict(selector=" ", 
             props=[("margin","0"),
                    ("font-family",'"Helvetica", "Arial", sans-serif'),
                    ("border-collapse", "collapse"),
                    ("border", "none"),
                    ("border","1px solid #ddd")
                       ]),

        #header color - optional
        dict(selector="thead", 
             props=[("background-color","#fff")
                   ]),

        #background shading
        dict(selector="tbody tr:nth-child(even)",
             props=[("background-color", "#cde")]),
        dict(selector="tbody tr:nth-child(odd)",
             props=[("background-color", "#eee")]),

        #cell spacing
        dict(selector="td", 
             props=[("padding", "1.0em")]),

        #header cell properties
        dict(selector="th", 
             props=[("font-size", "100%"),
                    ("text-align", "center")]),


    ]
    return (df.style.set_table_styles(styles).format({col: "{:.2f}" for col in df.columns \
                                                      if assembled_df.dtypes[col] == 'float64'}))

# Search Example

In [8]:
look_cols = list(field_search('fed') - (field_search('pos_') - field_search('pos_1')))
look_cols

['pos_1_fed_gleens',
 'pos_1_fed_2pw',
 'pos_1_fed_2o',
 'pos_1_fed_qic',
 'pos_1_adv_tech_taken_3vp_per_fed_pass',
 'pos_1_adv_tech_taken_5vp_per_fed',
 'pos_1_fed_2k',
 'pos_1_fed_6c',
 'pos_1_feds_taken',
 'pos_1_fed_12vp',
 'pos_1_score_fed']

# Basic Data Cleaning

In [9]:
# Fields with one column for each player
PLAYER_DROP_FIELDS = ['_research_level_dip']

# Fields with only one column for the table
SINGLE_DROP_FIELDS = ['Unnamed: 0']

drop_col_list = []
for field in PLAYER_DROP_FIELDS:
    drop_col_list += all_players(field)
drop_col_list += SINGLE_DROP_FIELDS
df_cleaned = df_all.drop(columns=drop_col_list,errors='ignore')

In [10]:
print('Sample size after cleaning', len(df_cleaned))

Sample size after cleaning 6714


# Filtering

In [11]:
VARIANTS = ['standard'] # Possible values: ['standard','more-balanced','beta']

MAP_LAYOUTS = ['standard'] # Possible to include 'xshape', but doesn't make sense for 4p

MIN_AVG_ELO = 200

# 4 player games only
df_filtered = df_cleaned.loc[df_cleaned.num_players == 4]

# Filter variants
df_filtered = df_filtered.loc[df_filtered.balance_variant.isin(VARIANTS)]

# Filter map layout
df_filtered = df_filtered.loc[df_filtered.map_layout.isin(MAP_LAYOUTS)]

# Remove games where any player is dropped
for col in all_players('_dropped'):
    df_filtered = df_filtered.loc[(df_filtered[col] == False)]

In [12]:
print('Sample size after basic filtering', len(df_filtered))

Sample size after basic filtering 4323


In [13]:
# Min ELO filter
df_filtered = df_filtered.loc[df_filtered['average_elo'] >= MIN_AVG_ELO]

In [14]:
print('Sample size after ELO filtering', len(df_filtered))

Sample size after ELO filtering 2514


# R1 Actions

In [15]:
action_names = {
    "2o_action": "power3",
    "7c_action": "power4",
    "2k_action": "power5",
    "1terra_action": "power6",
    "2terra_action": "power2",
    "2pt_action": "power7",
    "3k_action": "power1",
    "4q_action": "qic1",
    "3q_action": "qic2",
    "2q_action": "qic3"
}

In [16]:
faction_list = df_cleaned.pos_1_faction.unique()
assert(len(faction_list) == 14)

In [17]:
# look_cols = list(field_search('action_r_6') - (field_search('pos_') - field_search('pos_1')))
# look_cols

In [22]:
pd.options.display.float_format = '{:.2}'.format
for i in [6,5,4,3,2,1]:
    for action in action_names.keys():
        full_dict = {}
        sort_key = '% took {} in R{}'.format(action,i)
        for faction in faction_list:
            faction_dict = {}
            faction_df = get_faction_df(df_filtered, faction)
            take_df = faction_df[faction_df['target_action_r_{}_{}'.format(i,action)] == 1]
            no_take_df = faction_df[faction_df['target_action_r_{}_{}'.format(i,action)] != 1]
            faction_dict[sort_key] = len(take_df)/len(faction_df) * 100
            faction_dict['+pts if take {} vs no take in R{}'.format(action,i)] = take_df.target_score.mean() - no_take_df.target_score.mean()
            faction_dict['avg pts if take {} in R{}'.format(action,i)] = take_df.target_score.mean()
            faction_dict['avg pts if no take {} in R{}'.format(action,i)] = no_take_df.target_score.mean()
            faction_dict['overall avg score'] = faction_df.target_score.mean()
            faction_dict['total games'] = faction_df.shape[0]
            full_dict[faction] = faction_dict
        assembled_df = pd.DataFrame.from_dict(full_dict,orient='index').sort_values(by=sort_key, ascending=False)
        html_df = getTableHTML(assembled_df)
        
        path = 'plots/actions_analysis/'
        if not os.path.exists(path):
            os.makedirs(path)
        with open(path + 'r{}_{}.html'.format(i,action), 'w') as fo:
            fo.write(html_df.to_html())
html_df

,% took 2q_action in R1,+pts if take 2q_action vs no take in R1,avg pts if take 2q_action in R1,avg pts if no take 2q_action in R1,overall avg score,total games
ivits,0.11,-29.87,134.00,163.87,163.84,897
lantids,0.00,nan,nan,149.60,149.60,555
gleens,0.00,nan,nan,145.80,145.80,592
terrans,0.00,nan,nan,157.45,157.45,941
xenos,0.00,nan,nan,149.39,149.39,681
ambas,0.00,nan,nan,155.14,155.14,719
firaks,0.00,nan,nan,158.63,158.63,695
geodens,0.00,nan,nan,147.55,147.55,671
taklons,0.00,nan,nan,165.37,165.37,853
itars,0.00,nan,nan,166.09,166.09,918


In [19]:
pd.options.display.float_format = '{:.2}'.format
for i in [6,5,4,3,2,1]:
    full_dict = {}
    for faction in faction_list:
        faction_dict = {}
        faction_df = get_faction_df(df_filtered, faction)
        for action in action_names.keys():
            take_df = faction_df[faction_df['target_action_r_{}_{}'.format(i,action)] == 1]
            no_take_df = faction_df[faction_df['target_action_r_{}_{}'.format(i,action)] != 1]
            faction_dict['+pts if take {} vs no take in R{}'.format(action,i)] = take_df.target_score.mean() - no_take_df.target_score.mean()
            faction_dict['% took {} in R{}'.format(action,i)] = "{:.2f}".format(len(take_df)/len(faction_df) * 100) + '%'
        faction_dict['overall avg score'] = faction_df.target_score.mean()
        faction_dict['total games'] = faction_df.shape[0]
        full_dict[faction] = faction_dict
    assembled_df = pd.DataFrame.from_dict(full_dict,orient='index')
    html_df = getTableHTML(assembled_df)

    path = 'plots/actions_analysis/'
    if not os.path.exists(path):
        os.makedirs(path)
    with open(path + 'r{}_all_actions.html'.format(i), 'w') as fo:
        fo.write(html_df.to_html())
html_df

,+pts if take 2o_action vs no take in R1,% took 2o_action in R1,+pts if take 7c_action vs no take in R1,% took 7c_action in R1,+pts if take 2k_action vs no take in R1,% took 2k_action in R1,+pts if take 1terra_action vs no take in R1,% took 1terra_action in R1,+pts if take 2terra_action vs no take in R1,% took 2terra_action in R1,+pts if take 2pt_action vs no take in R1,% took 2pt_action in R1,+pts if take 3k_action vs no take in R1,% took 3k_action in R1,+pts if take 4q_action vs no take in R1,% took 4q_action in R1,+pts if take 3q_action vs no take in R1,% took 3q_action in R1,+pts if take 2q_action vs no take in R1,% took 2q_action in R1,overall avg score,total games
lantids,10.23,5.59%,10.33,32.07%,-1.10,12.25%,2.18,5.41%,-5.12,0.36%,-3.63,4.32%,nan,0.00%,-3.78,1.44%,nan,0.00%,nan,0.00%,149.60,555
gleens,0.77,15.71%,5.47,24.83%,-11.08,10.64%,-4.54,11.82%,-3.94,5.74%,1.46,14.19%,-4.81,0.17%,nan,0.00%,nan,0.00%,nan,0.00%,145.80,592
terrans,2.78,13.60%,6.67,20.83%,1.57,18.07%,-2.34,10.73%,-6.83,1.49%,-1.68,7.23%,3.11,2.44%,-3.16,2.34%,nan,0.00%,nan,0.00%,157.45,941
xenos,6.17,21.15%,-1.39,25.84%,-8.77,13.66%,-3.27,10.72%,-13.63,3.96%,7.04,6.46%,nan,0.00%,2.03,34.07%,nan,0.00%,nan,0.00%,149.39,681
ambas,-0.04,14.33%,3.93,25.03%,1.13,11.68%,1.48,24.06%,0.86,4.17%,0.08,2.50%,nan,0.00%,3.00,10.15%,nan,0.00%,nan,0.00%,155.14,719
firaks,9.50,35.25%,4.01,24.60%,-9.89,9.93%,-7.22,9.78%,-3.12,1.29%,0.62,1.15%,-0.64,0.14%,-9.05,1.58%,nan,0.00%,nan,0.00%,158.63,695
geodens,0.64,12.07%,0.06,20.57%,3.93,19.52%,7.51,28.76%,8.50,15.80%,-17.37,0.89%,nan,0.00%,-11.25,1.79%,nan,0.00%,nan,0.00%,147.55,671
taklons,8.12,43.73%,7.89,32.24%,-2.04,18.87%,5.82,36.11%,-0.68,7.85%,-1.74,3.40%,2.76,1.17%,5.07,5.16%,nan,0.00%,nan,0.00%,165.37,853
ivits,2.98,49.28%,0.63,15.16%,0.04,16.61%,4.75,33.67%,-0.50,10.81%,-7.13,4.68%,-3.98,1.34%,7.30,1.90%,43.76,0.22%,-29.87,0.11%,163.84,897
itars,2.27,30.07%,0.00,18.41%,-5.26,5.45%,3.15,19.93%,-23.45,1.09%,9.33,31.70%,0.41,0.44%,-1.95,2.29%,nan,0.00%,nan,0.00%,166.09,918


# Faction Early Feds

In [20]:
faction_list = df_cleaned.pos_1_faction.unique()
assert(len(faction_list) == 14)

In [21]:
pd.options.display.float_format = '{:.2}'.format
for i in [3,2,1]:
    full_dict = {}
    sort_key = '% made fed in 5vp/fed round R{}'.format(i)
    for faction in faction_list:
        faction_dict = {}
        faction_df = get_faction_df(df_filtered, faction)
        fed_scoring = faction_df[faction_df['round_{}_scoring'.format(i)] == '5vp_per_fed_place']
        made_fed = faction_df[(faction_df['round_{}_scoring'.format(i)] == '5vp_per_fed_place') & (faction_df['target_score_round{}'.format(i)] >= 5)]
        no_fed = faction_df[(faction_df['round_{}_scoring'.format(i)] == '5vp_per_fed_place') & (faction_df['target_score_round{}'.format(i)] == 0)]
        faction_dict[sort_key] = len(made_fed)/len(fed_scoring) * 100
        faction_dict['if 5vp/fed scoring R{}, +pts for fed vs no fed'.format(i)] = made_fed.target_score.mean() - no_fed.target_score.mean()
        faction_dict['if 5vp/fed scoring R{}, +pts vs avg if made fed'.format(i)] = made_fed.target_score.mean() - fed_scoring.target_score.mean()
        faction_dict['+pts if 5vp/fed scoring R{}'.format(i)] = fed_scoring.target_score.mean() - faction_df.target_score.mean()
        faction_dict['overall avg score'] = faction_df.target_score.mean()
        faction_dict['avg score if 5vp/fed scoring R{}'.format(i)] = fed_scoring.target_score.mean()
        faction_dict['avg score if made fed in 5vp/fed scoring R{}'.format(i)] = made_fed.target_score.mean()
        faction_dict['% players pick if 5vp/fed scoring R{}'.format(i)] = ((fed_scoring.shape[0]/faction_df.shape[0])*100)
        faction_dict['total games'] = faction_df.shape[0]
        faction_dict['games with 5vp/fed scoring R{}'.format(i)] = fed_scoring.shape[0]
        faction_dict['games fed made in 5vp/fed scoring R{}'.format(i)] = made_fed.shape[0]
        full_dict[faction] = faction_dict
    assembled_df = pd.DataFrame.from_dict(full_dict,orient='index').sort_values(by=sort_key, ascending=False)
    html_df = getTableHTML(assembled_df)
    
    path = 'plots/early_fed_analysis/'
    if not os.path.exists(path):
        os.makedirs(path)
    with open(path + 'r{}_fed_details.html'.format(i), 'w') as fo:
        fo.write(html_df.to_html())
html_df

,% made fed in 5vp/fed round R1,"if 5vp/fed scoring R1, +pts for fed vs no fed","if 5vp/fed scoring R1, +pts vs avg if made fed",+pts if 5vp/fed scoring R1,overall avg score,avg score if 5vp/fed scoring R1,avg score if made fed in 5vp/fed scoring R1,% players pick if 5vp/fed scoring R1,total games,games with 5vp/fed scoring R1,games fed made in 5vp/fed scoring R1
ivits,75.00,14.21,3.55,-1.59,163.84,162.25,165.80,12.04,897,108,81
gleens,16.42,-20.72,-17.32,-12.03,145.80,133.78,116.45,11.32,592,67,11
lantids,13.73,5.73,4.94,-1.97,149.60,147.63,152.57,9.19,555,51,7
taklons,2.41,-0.31,-0.31,-2.57,165.37,162.81,162.50,9.73,853,83,2
geodens,1.64,30.65,30.15,-4.70,147.55,142.85,173.00,9.09,671,61,1
nevlas,1.61,24.25,23.85,2.48,157.67,160.15,184.00,10.51,590,62,1
xenos,1.49,-24.05,-23.69,-6.70,149.39,142.69,119.00,9.84,681,67,1
terrans,0.00,nan,nan,1.84,157.45,159.28,nan,11.26,941,106,0
ambas,0.00,nan,nan,-2.77,155.14,152.37,nan,9.74,719,70,0
firaks,0.00,nan,nan,-8.87,158.63,149.77,nan,10.50,695,73,0
